## Tutorial for the Intended Usage of the Feature Extraction Pipeline API

##### Please ensure that you have spaCy and the "es_core_news_md" pipeline installed

In [1]:
# !python -m spacy download es_core_news_md

In [2]:
import pprint
from utils import read_corpus
from features import feature_pipeline

# Setup pretty printer
p = pprint.PrettyPrinter(indent=4, width=140)

### Introduction
This notebook is intended to illustrate and explain the main workflows possible when using the feature extraction pipeline API for computing statistical/numerical features from a raw text in our collected corpus of Spanish texts. This notebook should be treated as a secondary resource for understanding the API, the primary resource being the file `features.py` containing the source code.

Let's begin by broadly describing the stages of the feature extraction pipeline. To calculate any statistical feature from a raw, unprocessed text, the following steps must occur in sequence:
1. Initialize the pipeline object
2. Pre-process (clean up) the text to remove stray whitespaces, numerals, characters, etc.
3. Extract fundamental attributes of the text (eg., tokens, POS tags, lemmas, etc.) using spaCy
4. Use the extracted attributes to calculate statistical features of the text (eg., total number of tokens, type-token ratio, pronoun density, etc.)

It is not possible to calculate a statistical feature without first extracting the fundamental attributes necessary for calculating that feature. In other words, we cannot reach stage 4 without completing stages 1, 2 and 3. However, the API is written with some shortcuts in place, allowing us to almost never need to explicitly call the `.preprocess()` method in our code. Depending on our usage we can even sometimes skip writing the spaCy step. It is most crucial to remember that the unprocessed text must be passed as an argument at some stage within the pipeline, but it is quite flexible as to which stage that should be.

Through this tutorial we will understand these shortcuts and learn how to best apply them in workflows.

### Corpus Reading
Let's begin by loading the corpus and seeing a broad view of what it looks like:

In [3]:
corpus = read_corpus()

for k, v in corpus.items():
    print(f"{k}: {len(v)}")

A1: 94
A2: 62
B2: 110
B1: 42


Let's pick a text from the corpus for the purposes of this demo:

In [4]:
un_processed_text = corpus["A1"][80]["content"]
print(un_processed_text)

15. EL INVIERNO
El invierno es la estación fría. En el invierno
los días son muy cortos y las noches son
muy largas. Cuando hace mucho frío el agua
se hiela y cae nieve. En la zona tórrida no hay
hielo ni nieve, y hace siempre calor. En las
zonas templadas no hay hielo ni nieve sino en
el invierno. En las zonas glaciales hay hielo y
nieve durante todas las estaciones.
Los muchachos están alegres cuando hiela y
cae la nieve. Entonces patinan en los ríos y en
los lagos helados. Hacen pelotas de nieve y se
las arrojan unos a otros y juegan a la guerra.
Hacen también imágenes de nieve. Cuando ha
caído la nieve los muchachos traen sus trineos
sobre la nieve.
Cuando deshiela y la nieve desaparece, los
niños están muy tristes porque no pueden
patinar ni jugar más con pelotas de nieve.
Pero los pobres están muy alegres, porque
durante el invierno hace demasiado frío para
ellos. En la primavera ellos no tienen frío.
El invierno dura desde el veintiuno de diciembre
hasta el veintiuno de marzo.



We can see that this piece of text is formatted somewhat "irregularly". It is a poem, so it has line breaks in the middle of grammatical sentences. It also has a title at the top, which is not a necessary component of the content of the text. In order to derive syntactic attributes like POS tags and dependency parses we will need to convert this text into a more standard form that can be interpreted by spaCy.

### Text Preprocessing
Let's create a pipeline for cleaning up this text and extracting important attributes and features from it.

In [5]:
# This step passes the un-processed text to the pipeline and automatically cleans it up using the .preprocess() method
pipe = feature_pipeline(un_processed_text)
print(pipe.text)

el invierno el invierno es la estación fría. en el invierno los días son muy cortos y las noches son muy largas. cuando hace mucho frío el agua se hiela y cae nieve. en la zona tórrida no hay hielo ni nieve, y hace siempre calor. en las zonas templadas no hay hielo ni nieve sino en el invierno. en las zonas glaciales hay hielo y nieve durante todas las estaciones. los muchachos están alegres cuando hiela y cae la nieve. entonces patinan en los ríos y en los lagos helados. hacen pelotas de nieve y se las arrojan unos a otros y juegan a la guerra. hacen también imágenes de nieve. cuando ha caído la nieve los muchachos traen sus trineos sobre la nieve. cuando deshiela y la nieve desaparece, los niños están muy tristes porque no pueden patinar ni jugar más con pelotas de nieve. pero los pobres están muy alegres, porque durante el invierno hace demasiado frío para ellos. en la primavera ellos no tienen frío. el invierno dura desde el veintiuno de diciembre hasta el veintiuno de marzo.


The text looks much more standardized now, which makes it easier for downstream functions to extract things from it in a consistent manner.

In the above cell we only called the pipeline object constructor but it automatically gave us a cleaned text. That is because the constructor already contains a call for the `.preprocess()` method if a text has been supplied to the object. This behaviour is functionally equivalent to the following code:

In [6]:
pipe = feature_pipeline()
cleaned_text = pipe.preprocess(un_processed_text)
print(cleaned_text)

el invierno el invierno es la estación fría. en el invierno los días son muy cortos y las noches son muy largas. cuando hace mucho frío el agua se hiela y cae nieve. en la zona tórrida no hay hielo ni nieve, y hace siempre calor. en las zonas templadas no hay hielo ni nieve sino en el invierno. en las zonas glaciales hay hielo y nieve durante todas las estaciones. los muchachos están alegres cuando hiela y cae la nieve. entonces patinan en los ríos y en los lagos helados. hacen pelotas de nieve y se las arrojan unos a otros y juegan a la guerra. hacen también imágenes de nieve. cuando ha caído la nieve los muchachos traen sus trineos sobre la nieve. cuando deshiela y la nieve desaparece, los niños están muy tristes porque no pueden patinar ni jugar más con pelotas de nieve. pero los pobres están muy alegres, porque durante el invierno hace demasiado frío para ellos. en la primavera ellos no tienen frío. el invierno dura desde el veintiuno de diciembre hasta el veintiuno de marzo.


\
As a rule of thumb, it is easiest to pass the unprocessed text to the pipeline at initialization, since the constructor has the effect of resetting all of the attributes to empty lists, giving the pipeline a blank slate for processing a new text that may come its way.

### Extracting Attributes of the Text using SpaCy
Now that we have cleaned up the text into a standard form, spaCy will be able to derive some fundamental syntactic attributes from the text. Some attributes of the text that we might be interested in are the sentences, tokens and POS tags. We can try accessing them, but they won't be accessible at this stage since by default the pipeline constructor does not execute any of the spaCy methods.

In [7]:
# The below calls to access class attributes will just return empty lists since those items have not been extracted yet
print(pipe.sentences)
print(pipe.tokens)
print(pipe.pos_tags)

[]
[]
[]


These attributes must be extracted from the text using spaCy's Spanish pipeline. It is recommended that we generate the attribute lists as and when we need them, since extracting all of the attributes for every text can be a bit slow (although the pre-processing of the text is typically the slowest step in the pipeline). We can extract some attributes as follows:

In [8]:
pipe.get_sentences()  # populates the pipe.sentences attribute
pipe.get_tokens()  # populates the pipe.tokens attribute

# Print out the attributes that were populated
p.pprint(pipe.sentences)
print()
print(pipe.tokens)

[   'el invierno el invierno es la estación fría.',
    'en el invierno los días son muy cortos y las noches son muy largas.',
    'cuando hace mucho frío el agua se hiela y cae nieve.',
    'en la zona tórrida no hay hielo ni nieve, y hace siempre calor.',
    'en las zonas templadas no hay hielo ni nieve sino en el invierno.',
    'en las zonas glaciales hay hielo y nieve durante todas las estaciones.',
    'los muchachos están alegres cuando hiela y cae la nieve.',
    'entonces patinan en los ríos y en los lagos helados.',
    'hacen pelotas de nieve y se las arrojan unos a otros y juegan a la guerra.',
    'hacen también imágenes de nieve.',
    'cuando ha caído la nieve los muchachos traen sus trineos sobre la nieve.',
    'cuando deshiela y la nieve desaparece, los niños están muy tristes porque no pueden patinar ni jugar más con pelotas de nieve.',
    'pero los pobres están muy alegres, porque durante el invierno hace demasiado frío para ellos.',
    'en la primavera ellos no 

\
Calling the methods above will populate the respective attributes with lists, but they also return the lists as outputs. We can assign the output to a variable and access it that way as well.

In [9]:
tags = pipe.get_pos_tags()
print(tags == pipe.pos_tags)  # check if they are the same
print(tags)

True
['DET', 'NOUN', 'DET', 'NOUN', 'AUX', 'DET', 'NOUN', 'ADJ', 'PUNCT', 'ADP', 'DET', 'NOUN', 'DET', 'NOUN', 'VERB', 'ADV', 'ADJ', 'CCONJ', 'DET', 'NOUN', 'VERB', 'ADV', 'ADJ', 'PUNCT', 'SCONJ', 'AUX', 'DET', 'NOUN', 'DET', 'NOUN', 'PRON', 'VERB', 'CCONJ', 'VERB', 'NOUN', 'PUNCT', 'ADP', 'DET', 'NOUN', 'ADJ', 'ADV', 'AUX', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'CCONJ', 'AUX', 'ADV', 'NOUN', 'PUNCT', 'ADP', 'DET', 'NOUN', 'ADJ', 'ADV', 'AUX', 'NOUN', 'CCONJ', 'NOUN', 'CCONJ', 'ADP', 'DET', 'NOUN', 'PUNCT', 'ADP', 'DET', 'NOUN', 'ADJ', 'AUX', 'NOUN', 'CCONJ', 'NOUN', 'ADP', 'DET', 'DET', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'VERB', 'ADJ', 'SCONJ', 'ADJ', 'CCONJ', 'VERB', 'DET', 'NOUN', 'PUNCT', 'ADV', 'AUX', 'ADP', 'DET', 'NOUN', 'CCONJ', 'ADP', 'DET', 'NOUN', 'ADJ', 'PUNCT', 'AUX', 'NOUN', 'ADP', 'NOUN', 'CCONJ', 'PRON', 'PRON', 'AUX', 'PRON', 'ADP', 'PRON', 'CCONJ', 'AUX', 'ADP', 'DET', 'NOUN', 'PUNCT', 'AUX', 'ADV', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'SCONJ', 'AUX', 'VERB', 'DET', 'NOUN', 'DET'

Putting it all together, let's create a new pipeline object and give it the text to automatically pre-process. We can then just call a `.get_*` method to access the attribute using spaCy, completely eliminating the explicit writing of the pre-processing step.

In [10]:
pipe = feature_pipeline(un_processed_text)
print(pipe.get_noun_chunks())

['el invierno', 'el invierno', 'la estación', 'el invierno', 'los días', 'las noches', 'frío', 'el agua', 'se', 'nieve', 'la zona', 'hielo', 'nieve', 'calor', 'las zonas', 'hielo', 'nieve', 'el invierno', 'las zonas', 'hielo', 'nieve', 'las estaciones', 'los muchachos', 'la nieve', 'los ríos', 'los lagos', 'pelotas', 'nieve', 'se', 'las', 'unos', 'otros', 'la guerra', 'imágenes', 'nieve', 'la nieve', 'los muchachos', 'sus trineos', 'la nieve', 'la nieve', 'los niños', 'pelotas', 'nieve', 'los pobres', 'el invierno', 'ellos', 'la primavera', 'ellos', 'frío', 'el invierno']


Alternatively, we could also create a blank pipeline object and pass in the text through the `.get_*` method. This is functionally the same as the above. The only difference is that the text pre-processing will occur at the spaCy stage instead of the constructor stage.

In [11]:
pipe = feature_pipeline()
print(pipe.get_noun_chunks(un_processed_text))

['el invierno', 'el invierno', 'la estación', 'el invierno', 'los días', 'las noches', 'frío', 'el agua', 'se', 'nieve', 'la zona', 'hielo', 'nieve', 'calor', 'las zonas', 'hielo', 'nieve', 'el invierno', 'las zonas', 'hielo', 'nieve', 'las estaciones', 'los muchachos', 'la nieve', 'los ríos', 'los lagos', 'pelotas', 'nieve', 'se', 'las', 'unos', 'otros', 'la guerra', 'imágenes', 'nieve', 'la nieve', 'los muchachos', 'sus trineos', 'la nieve', 'la nieve', 'los niños', 'pelotas', 'nieve', 'los pobres', 'el invierno', 'ellos', 'la primavera', 'ellos', 'frío', 'el invierno']


\
Here is a list of all of the spaCy features supported by our pipeline so far:
* sentences: \
    extraction function - `pipe.get_sentences()`, \
    attribute - `pipe.sentences`
* tokens: \
    extraction function - `pipe.get_tokens()`, \
    attribute - `pipe.tokens`
* lemmas: \
    extraction function - `pipe.get_lemmas()`, attribute - `pipe.lemmas`
* POS tags: \
    extraction function - `pipe.get_pos_tags()`, \
    attribute - `pipe.pos_tags`
* morphology tags: \
    extraction function - `pipe.get_morphology()`, \
    attribute - `pipe.morphs`
* dependency parses: \
    extraction function - `pipe.get_dependency_parses()`, \
    attribute - `pipe.parses`
* noun phrase chunks: \
    extraction function - `pipe.get_noun_chunks()`, \
    attribute - `pipe.noun_chunks`

What if we want to extract all of the spaCy features in one go, instead of calling each of the `.get_*` methods one by one? We can do that by calling the method `.full_spacy()` which will extract all of these features, OR we could initialize the pipeline object with the flag `full_spacy=True`.

In [12]:
pipe = feature_pipeline(un_processed_text, full_spacy=True)
# Equivalent to:
# pipe = feature_pipeline()
# pipe.full_spacy(un_processed_text)  # does not return any outputs, saves directly to attributes

# All of the following items will automatically be extracted as part of the spaCy pipeline:
print(pipe.text)
print()
print(pipe.parses)

# Commented out for brevity
# p.pprint(pipe.sentences)
# print(pipe.tokens)
# print(pipe.lemmas)
# print(pipe.pos_tags)
# print(pipe.morphs)
# print(pipe.noun_chunks)

el invierno el invierno es la estación fría. en el invierno los días son muy cortos y las noches son muy largas. cuando hace mucho frío el agua se hiela y cae nieve. en la zona tórrida no hay hielo ni nieve, y hace siempre calor. en las zonas templadas no hay hielo ni nieve sino en el invierno. en las zonas glaciales hay hielo y nieve durante todas las estaciones. los muchachos están alegres cuando hiela y cae la nieve. entonces patinan en los ríos y en los lagos helados. hacen pelotas de nieve y se las arrojan unos a otros y juegan a la guerra. hacen también imágenes de nieve. cuando ha caído la nieve los muchachos traen sus trineos sobre la nieve. cuando deshiela y la nieve desaparece, los niños están muy tristes porque no pueden patinar ni jugar más con pelotas de nieve. pero los pobres están muy alegres, porque durante el invierno hace demasiado frío para ellos. en la primavera ellos no tienen frío. el invierno dura desde el veintiuno de diciembre hasta el veintiuno de marzo.

['de

### Numerical/Statistical Feature Extraction
The most important aspect of the feature extraction pipeline is the ability to derive statistical/numerical features from the text given to it. For a comprehensive guide of all of the features that this pipeline is capable of computing please see the project report (TODO: LINK TO PROJECT REPORT). \
\
Using the pipeline that we created and the attributes that we extracted in the previous cell, here is how we can derive some features from a text using the pipeline:

In [13]:
num_tokens = pipe.num_tokens()  # internally accesses pipe.tokens
log_op_density = pipe.logical_operators()  # internally accesses pipe.tokens
fh_score, syls_per_sent = pipe.fernandez_huerta_score()  # internally accesses pipe.tokens and pipe.sentences

print(num_tokens)
print(log_op_density)
print(fh_score)
print(syls_per_sent)

200
0.06951871657754011
90.29653846153847
21.866666666666667


It is important to note that any of the statistical feature functions can be called directly without needing to run any of the spaCy extractors first. As long as a feature pipeline object has been created, calling any of the feature functions will automatically extract the spaCy features necessary for computing the desired statistical feature. For example:

In [14]:
# Explicitly specifying full_spacy=False for demonstration purposes (default behaviour)
pipe = feature_pipeline(un_processed_text, full_spacy=False)
# The above step simply cleans up the text. No spaCy features are extracted

# All of these methods try to access their required spaCy attributes, and if
# they find that the attribute does not yet exist the necessary method will
# be called interally to generate those attributes.

num_tokens = pipe.num_tokens()  # internally accesses pipe.tokens
log_op_density = pipe.logical_operators()  # internally accesses pipe.tokens
fh_score, syls_per_sent = pipe.fernandez_huerta_score()  # internally accesses pipe.tokens and pipe.sentences

print(num_tokens)
print(log_op_density)
print(fh_score)
print(syls_per_sent)

200
0.06951871657754011
90.29653846153847
21.866666666666667


Alternatively, please note that the order of where the text is provided to the pipeline may also be switched. That is, the unprocessed text does not have to be provided to the pipeline at the initialization stage; it can be provided directly to the feature function as well. The text will automatically be cleaned up and the necessary attributes will be extracted using spaCy, following which the statistic will be calculated.

In [15]:
pipe = feature_pipeline()

# Saves the cleaned text to pipe.text and the extracted list of tokens to pipe.tokens
num_tokens = pipe.num_tokens(text=un_processed_text)
print(pipe.text)
print()
print(pipe.tokens)
print()

# No need to pass any arguments, it internally accesses pipe.tokens
log_op_density = pipe.logical_operators()

# No need to pass any arguments, it internally accesses pipe.tokens
# and extracts pipe.sentences from the cleaned up pipe.text
fh_score, syls_per_sent = pipe.fernandez_huerta_score()

print(num_tokens)
print(log_op_density)
print(fh_score)
print(syls_per_sent)

el invierno el invierno es la estación fría. en el invierno los días son muy cortos y las noches son muy largas. cuando hace mucho frío el agua se hiela y cae nieve. en la zona tórrida no hay hielo ni nieve, y hace siempre calor. en las zonas templadas no hay hielo ni nieve sino en el invierno. en las zonas glaciales hay hielo y nieve durante todas las estaciones. los muchachos están alegres cuando hiela y cae la nieve. entonces patinan en los ríos y en los lagos helados. hacen pelotas de nieve y se las arrojan unos a otros y juegan a la guerra. hacen también imágenes de nieve. cuando ha caído la nieve los muchachos traen sus trineos sobre la nieve. cuando deshiela y la nieve desaparece, los niños están muy tristes porque no pueden patinar ni jugar más con pelotas de nieve. pero los pobres están muy alegres, porque durante el invierno hace demasiado frío para ellos. en la primavera ellos no tienen frío. el invierno dura desde el veintiuno de diciembre hasta el veintiuno de marzo.

['el

Finally, let's extract all of the available statistical features in one go. Accomplishing this is as simple as creating a pipeline object and calling the `.feature_extractor()` method. We explicitly only write two lines, and the unprocessed text can be supplied to the pipeline at either line, but under the hood all 4 stages of processing the text will take place. \
(If the object is initialized with a text, `.feature_extractor()` does not require any arguments. Otherwise, if the object is initialized *without* a text, `.feature_extractor()` must be given a text in order to perform pre-processing and spaCy attribute extraction.)

In [16]:
pipe = feature_pipeline()
p.pprint(pipe.feature_extractor(un_processed_text))

# ALTERNATIVELY:
# pipe = feature_pipeline(un_processed_text)
# pipe.feature_extractor()

{   'ADJ': 0.065,
    'ADP': 0.11,
    'ADV': 0.065,
    'AUX': 0.075,
    'CCONJ': 0.07,
    'CONJ': 0.0,
    'CONTENT': 0.5769230769230769,
    'DET': 0.155,
    'EOL': 0.0,
    'FUNCTION': 0.4230769230769231,
    'INTJ': 0.01,
    'NOUN': 0.215,
    'NUM': 0.01,
    'PART': 0.0,
    'PRON': 0.035,
    'PROPN': 0.0,
    'PUNCT': 0.09,
    'SCONJ': 0.03,
    'SPACE': 0.0,
    'SYM': 0.0,
    'VERB': 0.07,
    'X': 0.0,
    'avg_ambiguation_all_words': 2.4411764705882355,
    'avg_ambiguation_content_words': 2.92,
    'avg_degree_of_abstraction': 7.193253968253968,
    'avg_rank_of_lemmas_in_freq_list': 560.565,
    'avg_sent_length': 13.333333333333334,
    'fernandez_huerta_score': 90.29653846153847,
    'logical_operator_density': 0.06951871657754011,
    'min_degree_of_abstraction': 5.0,
    'num_connectives': 8,
    'pronoun_density': 0.03626943005181347,
    'proportion_of_A_level_tokens': 0.5,
    'proportion_of_A_level_types': 0.3617021276595745,
    'syllables_per_sentence': 2